In [69]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from quantfreedom.enums import *
from tests.current_vid import SimpleBreakoutDynmaicLookback
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.simulate import run_df_backtest,or_backtest


from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
candles = dl_ex_candles(
    exchange="binance_us",
    symbol='BTCUSDT',
    timeframe='1h',
    since_date_ms=1626768000000,
    until_date_ms=1634745600000,
)

In [4]:
candles_to_df(candles)

,timestamp,open,high,low,close,volume
datetime,,,,,,
2021-07-20 08:00:00,1626768000000,29850.80,29877.37,29632.59,29632.59,10.693783
2021-07-20 09:00:00,1626771600000,29683.30,29797.07,29312.84,29453.24,18.397659
2021-07-20 10:00:00,1626775200000,29394.79,29697.53,29309.24,29675.77,12.609095
2021-07-20 11:00:00,1626778800000,29697.59,29787.75,29584.11,29701.90,3.852527
2021-07-20 12:00:00,1626782400000,29701.91,29793.72,29623.27,29693.78,4.089045
...,...,...,...,...,...,...
2021-10-20 11:00:00,1634727600000,63817.22,63924.46,63715.84,63790.05,3.989312
2021-10-20 12:00:00,1634731200000,63791.97,66787.05,63763.38,64382.64,59.154814
2021-10-20 13:00:00,1634734800000,64388.65,66059.71,64165.39,66031.42,55.343934


In [22]:
timestamp = candles[:, CandleBodyType.Timestamp]
open = candles[:, CandleBodyType.Open]
high = candles[:, CandleBodyType.High]
low = candles[:, CandleBodyType.Low]
close = candles[:, CandleBodyType.Close]

In [11]:
volatility_lb = 30
max_high_lb = 20

max_high = np.full_like(close, np.nan)
current_volatility = np.std(close[0:volatility_lb])

In [13]:
vol_lb_m_1 = volatility_lb - 1

for i in range(max(volatility_lb, max_high_lb), close.size):
    prev_vol = current_volatility
    current_volatility = np.std(close[i - vol_lb_m_1 : i + 1])

    delta_vol = (current_volatility - prev_vol) / current_volatility

    real_max_high_lb = max_high_lb * (1 + delta_vol)
    try:
        max_high[i] = high[max(i - int(real_max_high_lb), 0) : i].max()
    except:
        max_high[i] = high[i]

In [27]:
fig = go.Figure(
    data=[
        go.Candlestick(
            x=pd.to_datetime(timestamp, unit="ms"),
            open=open,
            high=high,
            low=low,
            close=close,
            name="candles",
        ),
        go.Scatter(
            x=pd.to_datetime(timestamp[~np.isnan(entry_prices)], unit="ms"),
            y=entry_prices[~np.isnan(entry_prices)],
            name="entries",
            mode="markers",
            marker=dict(color="yellow"),
        ),
    ]
)
fig.update_layout(height=800, xaxis_rangeslider_visible=False)
fig.show()

In [46]:
strategy = SimpleBreakoutDynmaicLookback(
    long_short="long",
    max_high_lb=np.array([30, 40]),
    volatility_lb=np.array([10, 20]),
)

In [48]:
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

In [61]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    since_date_ms=1697328000000,
    until_date_ms=1698105600000,
)

In [62]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

In [63]:
backtesting_settings = BacktestSettings()

In [64]:
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=100,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [65]:
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([6]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([6, 10]),
    sl_based_on_add_pct=np.array([0.1]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1]),
    trail_sl_when_pct=np.array([1]),
)

In [67]:
run_df_backtest(
    backtest_settings=backtesting_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    logger_bool=False,
    static_os=static_os,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 2
Total combinations of settings to test: 8
Total candles: 2,592
Total candles to test: 20,736


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,total_pnl,ending_eq
0,0,0,14.0,8,6,68.352,57.143,0.784,218.868,683.519,1683.519
1,0,1,14.0,8,6,99.311,57.143,0.748,253.796,993.109,1993.109
2,1,0,16.0,8,8,51.659,50.000,0.541,239.267,516.590,1516.590
3,1,1,16.0,8,8,78.653,50.000,0.513,276.065,786.533,1786.533
4,2,0,13.0,8,5,74.014,61.538,0.803,191.208,740.136,1740.136
5,2,1,13.0,7,6,84.022,53.846,0.696,195.896,840.222,1840.222
6,3,0,14.0,7,7,35.881,50.000,0.339,178.030,358.814,1358.814
7,3,1,14.0,7,7,59.864,50.000,0.336,204.108,598.642,1598.642


In [71]:
or_backtest(
    backtest_settings=backtesting_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    logger_bool=True,
    static_os=static_os,
    strategy=strategy,
    dos_index=0,
    ind_set_index=2,
    plot_results=True,
)

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,2,0,103,1697358900000,2023-10-15 08:35:00,EntryFilled,1000.000,932.978,7365.032,67.022,...,7432.054,26892.0,NaN,0.276,7432.054,NaN,0.200,26831.8,2.5,27575.8
1,2,0,104,1697359200000,2023-10-15 08:40:00,EntryFilled,1000.000,799.256,21960.768,200.744,...,7364.426,26893.0,NaN,0.550,14796.480,NaN,0.200,26831.8,2.6,27579.1
2,2,0,135,1697368500000,2023-10-15 11:15:00,StopLossFilled,940.131,940.131,NaN,NaN,...,NaN,NaN,26831.8,NaN,NaN,-59.869,NaN,NaN,NaN,NaN
3,2,0,162,1697376600000,2023-10-15 13:30:00,EntryFilled,940.131,893.231,3753.348,46.900,...,3800.248,26915.7,NaN,0.141,3800.248,NaN,0.600,26765.7,4.5,28138.2
4,2,0,193,1697385900000,2023-10-15 16:05:00,EntryFilled,940.131,783.090,14136.491,157.041,...,6739.936,26900.7,NaN,0.392,10540.184,NaN,0.400,26811.5,3.3,27800.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2,0,2577,1698101100000,2023-10-23 22:45:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.039,32596.4,NaN,NaN
107,2,0,2578,1698101400000,2023-10-23 22:50:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.046,32807.3,NaN,NaN
108,2,0,2579,1698101700000,2023-10-23 22:55:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.046,32825.8,NaN,NaN
109,2,0,2580,1698102000000,2023-10-23 23:00:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.047,32837.0,NaN,NaN
